In [ ]:
# Jeopardy Trivia App using CustomTkinter

import customtkinter as ctk
import csv
import getpass
import os
import tkinter as tk
from tkinter import messagebox, StringVar, OptionMenu
from datetime import datetime
import pandas as pd

# Load weekly questions and contestant data
questions = pd.read_csv('weekly_questions.csv')
contestants_df = pd.read_csv('weekly_contestants.csv')

# Sample list of names for the dropdown
# dropdown_names = ['Alina Lopez', 'Kelly Collins', 'Mia Malkova', 'Aria Lee', 'Natalia Starr']
dropdown_names = ['Isaac', 'Mark']

# Check if the current user has submitted answers
def has_submitted(user):
    return user in list(contestants_df['username'])

user_answers = {}

# Save answers to CSV
def save_answers(username, answers):
    answers['username'] = username
    answer_df = pd.DataFrame([answers], columns=contestants_df.columns)
    updated_df = pd.concat([contestants_df, answer_df], ignore_index=True)
    updated_df.to_csv('weekly_contestants.csv', index=False)

# Main App Class
class TriviaApp(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.title('Jeopardy Trivia')
        self.geometry('800x600')
        self.current_question = 0
        self.username = getpass.getuser()
        self.setup_home_page()

    def center_widget(self, widget):
        widget.pack(pady=10)

    def setup_home_page(self):
        self.home_frame = ctk.CTkFrame(self)
        self.home_frame.pack(expand=True, fill='both')

        title = ctk.CTkLabel(self.home_frame, text='Welcome to Jeopardy Trivia!', font=('Arial', 24))
        self.center_widget(title)

        self.selected_name = StringVar(value=self.username)
        username_dropdown = OptionMenu(self.home_frame, self.selected_name, *dropdown_names, command=self.update_username)
        username_dropdown.pack(anchor='ne', padx=10, pady=5)

        start_button = ctk.CTkButton(self.home_frame, text='Begin Trivia', command=self.begin_trivia)
        self.center_widget(start_button)

        leaderboard_button = ctk.CTkButton(self.home_frame, text='View Leaderboard', command=lambda: os.system("start https://www.youtube.com/watch?v=dQw4w9WgXcQ"))
        self.center_widget(leaderboard_button)

    def update_username(self, new_name):
        self.username = new_name

    def begin_trivia(self):
        if has_submitted(self.username):
            messagebox.showinfo('Error', 'You have already submitted an attempt for this week')
            self.home_frame.destroy()
            self.setup_home_page()
            return
        self.start_trivia()

    def start_trivia(self):
        self.home_frame.destroy()
        self.show_question()

    def show_question(self):
        if hasattr(self, 'question_frame'):
            self.question_frame.destroy()

        self.question_frame = ctk.CTkFrame(self)
        self.question_frame.pack(expand=True, fill='both')

        question_text = questions.loc[self.current_question]['question']
        self.question_label = ctk.CTkLabel(self.question_frame, text=question_text, font=('Arial', 18))
        self.center_widget(self.question_label)

        self.answer_entry = ctk.CTkEntry(self.question_frame)
        self.center_widget(self.answer_entry)

        if self.current_question == len(questions) - 1:
            submit_button = ctk.CTkButton(self.question_frame, text='Submit Answers', command=self.confirm_submission)
            self.center_widget(submit_button)
        else:
            next_button = ctk.CTkButton(self.question_frame, text='Next Question', command=self.save_answer)
            self.center_widget(next_button)

    def save_answer(self):
        answer = self.answer_entry.get().strip()
        if not answer:
            messagebox.showwarning('Warning', 'Please answer the question')
            return
        user_answers[f'q{self.current_question + 1}'] = answer
        self.current_question += 1
        self.show_question()

    def confirm_submission(self):
        if messagebox.askyesno('Submit', 'Are you sure you want to submit your answers?'):
            self.submit_answers()

    def submit_answers(self):
        self.save_answer
        save_answers(self.username, user_answers)
        messagebox.showinfo('Submit', 'Your answers have been submitted!')
        self.destroy()

if __name__ == '__main__':
    app = TriviaApp()
    app.mainloop()